In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/SNLP/comp0087_assignment-main

/content/drive/MyDrive/SNLP/comp0087_assignment-main


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.1 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, LEDForConditionalGeneration
import pandas as pd
from tqdm.notebook import tqdm
from pathlib import Path

In [ ]:
# Function for summarisation
def summarise(texts, tokenizer, model, device, min_summary_length, max_summary_length, gam=False):
    # tokenize the batch of texts
    inputs = tokenizer.batch_encode_plus(texts, return_tensors='pt', padding=True, truncation=True, max_length = 10000).to(device)

    if (gam == True):
      # global attention on the first token (cf. Beltagy et al. 2020)
      global_attention_mask = torch.zeros_like(inputs['input_ids'])
      global_attention_mask[:, 0] = 1
      # generate summaries for the batch of texts with global attention on the first token
      summary_ids = model.generate(inputs['input_ids'], attention_mask=global_attention_mask, num_beams=3, max_length=max_summary_length, min_length = min_summary_length)
    else:
      # generate summaries for the batch of texts
      summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=max_summary_length, min_length = min_summary_length)

    # decode the summary ids back into text
    batch_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    
    return batch_summaries

In [ ]:
# Move to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
# Setup the model and tokenizer
# model = LEDForConditionalGeneration.from_pretrained("allenai/led-large-16384").to(device)
# tokenizer = AutoTokenizer.from_pretrained("allenai/led-large-16384")
model = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384").to(device)
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

# Make folder if necessary
Path("data/echr/led_longformer").mkdir(parents=True, exist_ok=True)

cuda


In [ ]:
dff = pd.read_pickle(f"data/echr/led_longformer/non-anon_test.pkl")
dff

,itemid,text,violated_articles,violated,non_violated_articles,all_articles,importance,inadmissible,date,docname,summary
0,001-157392,"The applicants were born in 1984, 1974 and 198...",[5],True,[],[5],3,False,2015,CASE OF NABIL AND OTHERS v. HUNGARY,"), detention ordered under the immigration law..."
1,001-167110,The applicant was born in 1972 in Villahermosa...,[34],True,[],[34],3,False,2016,CASE OF CANO MOYA v. SPAIN,The applicant was born in 1972 in Villahermosa...
2,001-158944,The applicant was born in 1959 and is currentl...,[],False,[6],[6],4,False,2015,CASE OF SAZANOV v. RUSSIA,The applicant was born in 1959 and is currentl...
3,001-158865,The common feature of these three applications...,[],False,[],[],4,True,2015,CHAKKAS AND OTHERS v. CYPRUS,The common feature of these three applications...
4,001-172465,"The applicants were born in 1946, 1966 and 197...",[],False,[2],[2],4,False,2017,CASE OF BOROJEVIĆ AND OTHERS v. CROATIA,The applicants’ names were not given. The appl...
...,...,...,...,...,...,...,...,...,...,...,...
2993,001-153768,The applicant was born in 1973 and is currentl...,[],False,[5],[5],4,False,2015,CASE OF GAWRECKI v. POLAND,
2994,001-166930,The applicant was born in 1955. He is currentl...,"[13, 3]",True,[],"[13, 3]",4,False,2016,CASE OF PISKUNOV v. RUSSIA,
2995,001-158242,"The applicant, Mr Vladas Koturenka, is a Lithu...",[],False,[],[],4,True,2015,KOTURENKA v. LITHUANIA,
2996,001-184003,The list of applicants and the relevant detail...,[3],True,[],[3],4,False,2018,CASE OF KAYMAK AND OTHERS v. ROMANIA,


In [ ]:
# Loop through all text in data
for set_ in ["train","valid", "test"]:
    for name in ["non-anon"]:
    # for name in ["anon"]:
        print(f"{name} {set_} set")
        df = pd.read_pickle(f"data/echr/led_longformer/{name}_{set_}.pkl")
        if "summary" not in df.columns:
            df["summary"] = ""
        i=0
        batch_size = 2
        num_batches = (len(df)//batch_size)+1

        # Loop through summarise
        for j in tqdm(range(2500,num_batches)):
            start_idx = j*batch_size
            end_idx = (j+1)*batch_size
            texts = df['text'][start_idx:end_idx].tolist()
            batch_summaries = summarise(texts, tokenizer, model, device, min(256, min(df['text'][start_idx:end_idx].str.split().apply(len))), 512)

            for k, summary in enumerate(batch_summaries):
                df["summary"][start_idx+k] = summary

            # Save every 50 summaries in case of crashes
            if (j % 50==0) and (j>0):
                df.to_pickle(f"data/echr/led_longformer/{name}_{set_}.pkl")
            
        # Save summary
        df.to_pickle(f"data/echr/led_longformer/{name}_{set_}.pkl")

non-anon train set


  0%|          | 0/1051 [00:00<?, ?it/s]

<ipython-input-12-69ca9462e06e>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["summary"][start_idx+k] = summary


ValueError: ignored

testing

In [ ]:
text = df['text'][41]

inputs = tokenizer.encode(text, return_tensors="pt", padding=True, truncation=True, max_length=10000).to(device)

# Global attention on the first token (cf. Beltagy et al. 2020)
global_attention_mask = torch.zeros_like(inputs)
global_attention_mask[:, 0] = 1

# Generate Summary
summary_ids = model.generate(inputs, num_beams=3, min_length=256, max_length=512)
# summary_ids = model.generate(inputs, global_attention_mask=global_attention_mask, num_beams=3, min_length=256, max_length=512)
# min_summary_length = min(256, min([len(text.split()) for text in texts]))
# summary_ids = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512, min_length = min_summary_length)

batch_summaries = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
# batch_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
# print(tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
print(batch_summaries)

 points of law and sentencing, as well as the appeal against confiscation. According to Section 3 of the Sea-Water and Fisheries Act, its provisions do not affect the right of navigation in the exclusive economic zone under the Norwegian Economic Zone Act 1976, and are subject to the limitations which follow from public international law and agreements with other States. In the proceedings the applicants referred to the following provisions of the Convention on the Law of the Sea, which entered into force in 1994 and which were ratified by Norway in 1996: Article 56 Rights, jurisdiction and duties of the coastal State in the exclusive economic zone "1. In the exclusive economic zone, the coastal State has: (a) sovereign rights for the purpose of exploring and exploiting, conserving and managing the natural resources, whether living or non-living, of the waters superjacent to the seabed and of the seabed and its subsoil, and with regard to other activities for the economic exploitation 

In [ ]:
batch_summaries

' points of law and sentencing, as well as the appeal against confiscation. According to Section 3 of the Sea-Water and Fisheries Act, its provisions do not affect the right of navigation in the exclusive economic zone under the Norwegian Economic Zone Act 1976, and are subject to the limitations which follow from public international law and agreements with other States. In the proceedings the applicants referred to the following provisions of the Convention on the Law of the Sea, which entered into force in 1994 and which were ratified by Norway in 1996: Article 56 Rights, jurisdiction and duties of the coastal State in the exclusive economic zone "1. In the exclusive economic zone, the coastal State has: (a) sovereign rights for the purpose of exploring and exploiting, conserving and managing the natural resources, whether living or non-living, of the waters superjacent to the seabed and of the seabed and its subsoil, and with regard to other activities for the economic exploitation

In [ ]:
df = pd.read_pickle(f"data/echr/led_longformer/non-anon_train.pkl")
i = 0
print(df['text'][i])
print(df['summary'][i])